In [144]:
from resnet import ResNet34, ResNet
import torch
import math
model = ResNet34(10)

In [134]:
def reset_weights(model: ResNet):
	print(model)
	if hasattr(model, 'reset_parameters'):
			print('has attribute')
			model.reset_parameters()

In [141]:
model2 = torch.jit.load('models/model_scripted.pt')

In [148]:
def reset_scripted_module_parameters(scripted_module):
		# Iterate over all named parameters in the scripted module
		for name, param in scripted_module.named_parameters(recurse=True):
				# Apply weight initialization
				if param.dim() > 1:
						torch.nn.init.kaiming_uniform_(param, a=math.sqrt(5))
				# Apply bias initialization
				elif param.dim() == 1:
						fan_in, _ = torch.nn.init._calculate_fan_in_and_fan_out(param.unsqueeze(0))
						bound = 1 / math.sqrt(fan_in)
						torch.nn.init.uniform_(param, -bound, bound)

		for name, buffer in scripted_module.named_buffers():
				if 'running_mean' in name or 'running_var' in name:
						buffer.fill_(0 if 'mean' in name else 1)

In [170]:
c = [child for child in model2.children()]
p = [param for param in c[0].parameters()]
p[0][0, 0, 0, 0]

tensor(-0.0697, grad_fn=<SelectBackward0>)

In [169]:
reset_scripted_module_parameters(model2)

In [103]:
children = [child for child in model.children()]
children[0].weight[0, 0, 0, 0]

tensor(-0.0486, grad_fn=<SelectBackward0>)

In [80]:
children = [child for child in children[0].children()]
children

[Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 Sequential()]

In [82]:
seq = children[-1]
seq.layers

AttributeError: 'Sequential' object has no attribute 'layers'

In [121]:
# inspired by https://stackoverflow.com/a/77474877.
# very hacky, don't love this solution
def reinitialize(model):
		for l in model.layers:
				if hasattr(l,'kernel_initializer'):
						if hasattr(l.kernel_initializer, 'seed'):
								l.kernel_initializer.__init__(seed=np.random.randint(1e6))
						l.kernel.assign(l.kernel_initializer(tf.shape(l.kernel)))
				if hasattr(l,'bias_initializer'):
						if hasattr(l.bias_initializer, 'seed'):
								l.bias_initializer.__init__(seed=np.random.randint(1e6))
						l.bias.assign(l.bias_initializer(tf.shape(l.bias)))
				if hasattr(l,'recurrent_initializer'):
						if hasattr(l.recurrent_initializer, 'seed'):
								l.recurrent_initializer.__init__(seed=np.random.randint(1e6))
						l.recurrent_kernel.assign(l.recurrent_initializer(tf.shape(l.recurrent_kernel)))

In [128]:
reinitialize(model)
model.weights[0][0, 0, 0, 0]

<tf.Tensor: shape=(), dtype=float32, numpy=0.0018577129>

In [1]:
from model_interfaces import ModelInterface, TensorFlowModel, TorchModel
from model_garden import ModelGarden

path = 'models/compiled.keras'
file_path, file_extension = path.split('.')
file_name = file_path.split('/')[-1]
if file_extension == 'keras':
	custom_model = TensorFlowModel.load_model(path)
elif file_extension == 'pt':
	custom_model = TorchModel.load_model(path)
else:
	raise ValueError(f'Unexpected file extension: {file_extension}')
